### python version 확인

In [1]:
import sys
print(sys.version)

3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]


### install bs & selenium

In [2]:
!pip install bs4
!pip install selenium

  Obtaining dependency information for bs4 from https://files.pythonhosted.org/packages/51/bb/bf7aab772a159614954d84aa832c129624ba6c32faa559dfb200a534e50b/bs4-0.0.2-py2.py3-none-any.whl.metadata


### 라이브러리 import

In [5]:
from bs4 import BeautifulSoup
import requests
import re
import time
import os
import sys
import urllib.request
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd

### url 수집

In [6]:
# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

# 버전에 상관 없이 os에 설치된 크롬 브라우저 사용
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(3)
# 버전에 상관 없이 os에 설치된 크롬 브라우저 사용


# Naver API key 입력
client_id = 'k9kbYZS4KZ54NuuEitRI' 
client_secret = 'sBiu_KktIH'


# selenium으로 검색 페이지 불러오기 #
urls_list = []
postdate = []
titles = []

# 검색어 입력
keword = "세탁기"
encText = urllib.parse.quote(keword)

# 검색을 끝낼 페이지 입력
end = 2
if end == "":
    end = 1
else:
    end = int(end)
print("\n 1 ~ ", end, "페이지 까지 크롤링을 진행 합니다")

# 한번에 가져올 페이지 입력
#display = input("\n한번에 가져올 페이지 개수를 입력해주세요.(기본값:10, 최대값: 100):")
#if display == "":
#    display = 10
#else:
#    display = int(display)
#print("\n한번에 가져올 페이지 : ", display, "페이지")


for start in range(end):
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText + "&start=" + str((start*10)+1) #+ "&display=" + str(display) # JSON 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        
        data = json.loads(response_body.decode('utf-8'))['items']
        for row in data:
            if('blog.naver' in row['link']):
                urls_list.append(row['link'])
                postdate.append(row['postdate'])
                title = row['title']
                # html태그제거
                pattern1 = '<[^>]*>'
                title = re.sub(pattern=pattern1, repl='', string=title)
                titles.append(title)
        time.sleep(2)
    else:
        print("Error Code:" + rescode)

/var/folders/17/dm84jvsn5r70_mj0jskzbgmw0000gn/T/ipykernel_74740/2418853134.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())



 1 ~  2 페이지 까지 크롤링을 진행 합니다


### 본문 및 제목 가져오기

In [7]:
from selenium.webdriver.chrome.service import Service

# 크롬 드라이버 경로를 명시적으로 지정
# chrome_driver_path = r'C:\Users\glara\chromedriver-win64\chromedriver.exe'  # 여기에 크롬 드라이버의 경로를 입력

# 웹드라이버 초기화
# driver = webdriver.Chrome(executable_path=chrome_driver_path)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}


contents = []
comments_texts = []
try:
    for i in urls_list:
        print(i)
        driver.get(i)
        time.sleep(3)  # 대기시간 변경 가능

        iframe = driver.find_element(By.ID , "mainFrame") # id가 mainFrame이라는 요소를 찾아내고 -> iframe임
        driver.switch_to.frame(iframe) # 이 iframe이 내가 찾고자하는 html을 포함하고 있는 내용

        source = driver.page_source
        html = BeautifulSoup (source, 'html.parser')

        # 검색결과 확인용
        # with open("Output.txt", "w") as text_file:
        #     text_file.write(str(html))
        
        # 기사 텍스트만 가져오기
        content = html.select("div.se-main-container")
        #  list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')
        content = content.replace('\n', '')
        content = content.replace('\u200b', '')
        contents.append(content)


    blog_df = pd.DataFrame({'date': postdate, 'title': titles, 'content': contents, 'url' : urls_list})
    blog_df.to_csv('blog.csv', index=False, encoding='utf-8-sig')
except:
    contents.append('error')
    blog_df = pd.DataFrame({'date': postdate, 'title': titles, 'content': contents, 'url' : urls_list})
    blog_df.to_csv('blog.csv', index=False, encoding='utf-8-sig')

https://blog.naver.com/hs790619/223735818604
https://blog.naver.com/dltmdgns2525/223818209419
https://blog.naver.com/silentforest_studio/223833712411
https://blog.naver.com/jumm0723/223792125616
https://blog.naver.com/aquamoon0216/223829560332
https://blog.naver.com/treasure68/223820564791
https://blog.naver.com/denis98/223825260520
https://blog.naver.com/egofkhk/223832802942
https://blog.naver.com/betty6649/223811567888
https://blog.naver.com/neorysg1/223830957346
https://blog.naver.com/ringto263/223787437755
https://blog.naver.com/showkingcar/223792735556
https://blog.naver.com/siono/223822958185
https://blog.naver.com/hihihim1/223824578139
https://blog.naver.com/gantandair/223810336605
https://blog.naver.com/jejuna17/223803433387
https://blog.naver.com/girl901/223826021225
https://blog.naver.com/4myallife/223811701584
https://blog.naver.com/smallview/223802068919
https://blog.naver.com/oda999/223797951108


In [8]:
blog_df

,date,title,content,url
0,20250123,영통구 하수구막힘 이의동 세탁기 하수구 막힘,[50m© NAVER Corp.더보기 /OpenStreetMap지도 데이터x© NA...,https://blog.naver.com/hs790619/223735818604
1,20250402,로보락 세탁기 건조기 일체형 관리 편한 H1 Lite 사용 후기,[ 로보락 세탁기 건조기 일체형관리 편한 H1 Lite 사용 후기취직을 한 후 본...,https://blog.naver.com/dltmdgns2525/223818209419
2,20250415,삼성전자 그랑데 AI 드럼 세탁기 24kg 블랙 케비어 1등급...,"[""이 포스팅은 네이버 쇼핑 커넥트 활동의 일환으로, 판매 발생 시 수수료를 제공받...",https://blog.naver.com/silentforest_studio/223...
3,20250312,평택에어컨청소 세탁기까지 한번에!,"[ 몇 년 전 새로운 집으로 이사한 뒤,여름이 오기 전 에어컨 청소는 필수가...",https://blog.naver.com/jumm0723/223792125616
4,20250411,세탁기 사용법 삼성 통돌이 세탁기 돌리는법,[ 세탁기 사용법 삼성 통돌이 세탁기 돌리는법© 쉬운살림 리빙달갑자기 온도가 올라가...,https://blog.naver.com/aquamoon0216/223829560332
5,20250403,드럼세탁기 냄새 주기적인 청소 필요해요,[ 드럼세탁기 냄새 주기적인 청소 필요해요 오늘도 어김없이 찾아온 빨래데이 날씨가 ...,https://blog.naver.com/treasure68/223820564791
6,20250407,드럼세탁기청소 냄새 이물질 청소까지,[살림백서 세탁조클리너 드럼세탁기청소 겸용 1BOX : 살림백서우리집 세탁기 안심 ...,https://blog.naver.com/denis98/223825260520
7,20250414,일반 통돌이 세탁기 사용법 설치 청소 간편!,[ 안녕하세요. 김센스입니다. 다들 아시겠지만 다둥이 가정인 김센스네는 빨래를 이...,https://blog.naver.com/egofkhk/223832802942
8,20250327,lg 드럼세탁기 청소 방법 락스 세탁기 통세척 통살균 세탁조...,[1. lg 드럼세탁기 청소 방법 락스 세탁기 통세척 통살균 세탁조 고무패킹초보주부...,https://blog.naver.com/betty6649/223811567888
9,20250412,인천 세탁기 건조기 직렬설치 불가 단차 받침 공사 십정뜨란채,[인천 '십정뜨란채주공 1단지' 아파트 세탁기 건조기 직렬설치 불가 현상으로 단차 ...,https://blog.naver.com/neorysg1/223830957346
